In [1]:
import os
import torch
import csv
import numpy as np
import lightning as L
import data_utils.augmentations as augs
from data_utils.datamodules import AlignedMicroDataModule
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor

import sys
sys.path.append('..')
from alignment import alignment_utils as utils
from alignment.AlignCCA import AlignCCA
from models import Seq2SeqRNN

# Data Prep

In [2]:
def folds_exist(path, n_folds):
    for i in range(n_folds):
        f = os.path.join(path, f"fold_data/fold_{i}.h5")
        if not os.path.exists(f):
            print("❌ Missing:", f)
            return False
    return True

In [3]:
data_path = '../data/pt_decoding_data_S62.pkl'
fold_data_path = '../data/training_data_pooled'
patient_id = 'S14' # Training on S14's data.
batch_size = 500
n_folds = 20
val_size = 0.1

In [ ]:
data_filename = os.path.expanduser(data_path)
pt_data = utils.load_pkl(data_filename)

phoneme_index = 1 # All samples center around phoneme 1 onset.
lab_type = 'phon'
algn_type = 'phon_seq'
tar_data, pre_data = utils.decoding_data_from_dict(pt_data, patient_id, phoneme_index,
                                                   lab_type=lab_type,
                                                   algn_type=algn_type)
# tar_data = X1, y1, [y1, y2, y3], on the target patient
# pre_data = 7 * tar_data, on the 7 non-target patients
# pre_data is not used at all in training

augmentations = [augs.time_shifting, augs.noise_jitter, augs.scaling]
data = torch.Tensor(tar_data[0])
labels = torch.Tensor(tar_data[1]).long().unsqueeze(1) - 1
align_labels = torch.Tensor(tar_data[2]).long() - 1
pool_data = [(torch.Tensor(p[0]), torch.Tensor(p[2]).long() - 1, torch.Tensor(p[2]).long() - 1) for p in pre_data] 


In [6]:
if folds_exist(fold_data_path, n_folds):
    print("✅ All folds found, reusing existing DataModule...")
    dm = AlignedMicroDataModule(
        data, align_labels, align_labels, pool_data, AlignCCA,
        batch_size=batch_size, folds=n_folds, val_size=val_size,
        augmentations=augmentations, data_path=fold_data_path
    )
else:
    print("⚙️ Generating new folds...")
    dm = AlignedMicroDataModule(
        data, align_labels, align_labels, pool_data, AlignCCA,
        batch_size=batch_size, folds=n_folds, val_size=val_size,
        augmentations=augmentations, data_path=fold_data_path
    )
    dm.setup()

❌ Missing: ../data/training_data_pooled/fold_data/fold_4.h5
⚙️ Generating new folds...
DEBUG labels.shape: torch.Size([144, 3])
DEBUG first 5 rows of labels:
 tensor([[8, 0, 4],
        [7, 1, 4],
        [2, 5, 3],
        [1, 7, 0],
        [1, 8, 0]])


KeyboardInterrupt: 

# Train

In [9]:
gclip_val = 0.5
fs = 200
in_channels = data.shape[-1] # 111
num_classes = 9
kernel_time = 50  # ms
kernel_size = int(kernel_time * fs / 1000)  # kernel length in samples
stride_time = 50  # ms
stride = int(stride_time * fs / 1000)  # stride length in samples
padding = 0
n_enc_layers = 2
n_dec_layers = 1
activ = False
model_type = 'gru'

max_epochs = 500
log_dir = os.path.join("..", "data", "transformer_logs")
context_prefix = "pooled"

In [10]:
n_filters = 100
hidden_size = 500
cnn_dropout = 0.3
rnn_dropout = 0.3
learning_rate = 1e-4
l2_reg = 1e-5
n_iters = 20

**Model Architecture:**

1. Temporal Conv:
    - Input: 200 timesteps * 111 features
    - Output: 20 timesteps * 100 features
    - kernel (10 timesteps * 111 features) * 100 different versions of kernel
2. Encoder:
    - Input: 20 tokens * 100-dim vector per token
    - Output: h_20, a 500-dim vector
    - h_t = GRU(h_t-1, x_t)
        * t = 20 tokens
        * x_t = one token of 100-dim (features representing the current chunk of time)
        * h_t = hidden state of 500-dim (a new set of features to represent the current + prev chunk of time)
3. Decoder:
    - Input: encoder's output, a 500-dim vector encoding the whole sequence
    - Output: y_1, y_2, y_3, 3 classification results
    - h_t = GRU(h_t-1, x_t)
        * t = 3, each producing 1 phoneme prediction
        * h_t = hidden state of 500-dim (features representing current chunk of time + previous phon prediction)
        * x_t = a 500-dim representation of the predicted phon (x_0 = a sequence for class 0 "SOS")
    - y_t = integer in 0-8 = a_simple_proj(h_t)
    - x_t+1 = Embedding(y_t)

In [8]:
def data_prep(data_path = '../data/pt_decoding_data_S62.pkl', 
              fold_data_path = '../data/training_data_pooled', 
              patient_id = 'S14', 
              batch_size = 500, 
              n_folds = 20, 
              val_size = 0.1):
    
    data_filename = os.path.expanduser(data_path)
    pt_data = utils.load_pkl(data_filename)

    phoneme_index = 1 # All samples center around phoneme 1 onset.
    lab_type = 'phon'
    algn_type = 'phon_seq'
    tar_data, pre_data = utils.decoding_data_from_dict(pt_data, patient_id, phoneme_index,
                                                    lab_type=lab_type,
                                                    algn_type=algn_type)

    augmentations = [augs.time_shifting, augs.noise_jitter, augs.scaling]
    data = torch.Tensor(tar_data[0])
    labels = torch.Tensor(tar_data[1]).long().unsqueeze(1) - 1 
    align_labels = torch.Tensor(tar_data[2]).long() - 1
    pool_data = [(torch.Tensor(p[0]), torch.Tensor(p[2]).long() - 1, torch.Tensor(p[2]).long() - 1) for p in pre_data]

    if folds_exist(fold_data_path, n_folds):
        print("✅ All folds found, reusing existing DataModule...")
        dm = AlignedMicroDataModule(
            data, align_labels, align_labels, pool_data, AlignCCA,
            batch_size=batch_size, folds=n_folds, val_size=val_size,
            augmentations=augmentations, data_path=fold_data_path
        )
    else:
        print("⚙️ Generating new folds...")
        dm = AlignedMicroDataModule(
            data, align_labels, align_labels, pool_data, AlignCCA,
            batch_size=batch_size, folds=n_folds, val_size=val_size,
            augmentations=augmentations, data_path=fold_data_path
        )
        dm.setup()
    
    torch.save(
        {"patient_id": patient_id, 
         "n_folds": n_folds, 
         "dm": dm, 
         "data": torch.Tensor(tar_data[0])}, 
         f"{fold_data_path}/{patient_id}_prep.pt")

    pass

In [11]:
!python CLI.py data_prep --data_path="../data/pt_decoding_data_S62.pkl" --fold_data_path="../data/training_data_pooled" --n_folds=20 --batch_size=32 --val_size=0.2 

DEBUG: Current working directory: /Users/wangmaidou/Documents/EEG大模型/BCI Code/seq2seq_RNN/nn_models
DEBUG: data_path: ../data/pt_decoding_data_S62.pkl
DEBUG: fold_data_path: ../data/training_data_pooled
DEBUG: patient_id: S14
DEBUG: data_filename: ../data/pt_decoding_data_S62.pkl
DEBUG: File exists: True
❌ Missing: ../data/training_data_pooled/fold_data/fold_0.h5
⚙️ Generating new folds...
DEBUG labels.shape: torch.Size([144, 3])
DEBUG first 5 rows of labels:
 tensor([[8, 0, 4],
        [7, 1, 4],
        [2, 5, 3],
        [1, 7, 0],
        [1, 8, 0]])
^C
Traceback (most recent call last):
  File "/Users/wangmaidou/Documents/EEG大模型/BCI Code/seq2seq_RNN/nn_models/CLI.py", line 190, in <module>
    fire.Fire({
  File "/opt/anaconda3/envs/bci/lib/python3.12/site-packages/fire/core.py", line 135, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/bc

In [9]:
data_prep('../data/pt_decoding_data_S62.pkl', '../data/training_data_pooled', 'S14', 500, 20, 0.1)

❌ Missing: ../data/training_data_pooled/fold_data/fold_7.h5
⚙️ Generating new folds...
DEBUG labels.shape: torch.Size([144, 3])
DEBUG first 5 rows of labels:
 tensor([[8, 0, 4],
        [7, 1, 4],
        [2, 5, 3],
        [1, 7, 0],
        [1, 8, 0]])


KeyboardInterrupt: 

In [11]:
# specify a folder to save all accuracy data
acc_dir = os.path.join("..", "data", "accuracy_data_post_training")
iter_accs = []  # store all accuracies across iterations

for i in range(n_iters):
    print(f'##### Setting up data module for iteration {i+1} #####')
    dm.setup()  # prepare dataset (splits, etc.)
    
    fold_accs = []  # store results for this iteration’s folds

    # go through each fold
    for fold in range(n_folds):
        dm.set_fold(fold)
        in_channels = dm.get_data_shape()[-1]  # number of input features (111 channels)

        # build a fresh Seq2SeqRNN for this fold
        model = Seq2SeqRNN(in_channels, n_filters, hidden_size, num_classes, n_enc_layers,
                           n_dec_layers, kernel_size, stride, padding, cnn_dropout, rnn_dropout, model_type,
                           learning_rate, l2_reg, activation=activ, decay_iters=max_epochs)

        # callbacks: save best model + log learning rate
        callbacks = [
            ModelCheckpoint(monitor='val_acc', mode='max'),
            LearningRateMonitor(logging_interval='epoch'),
        ]

        # trainer controls training loop
        trainer = L.Trainer(default_root_dir=log_dir,
                            max_epochs=max_epochs,
                            gradient_clip_val=gclip_val,   # prevent exploding gradients
                            accelerator='auto',            # GPU if available
                            callbacks=callbacks,
                            logger=True,
                            enable_model_summary=False,
                            enable_progress_bar=False)

        # train and validate for all epochs
        trainer.fit(model=model, 
                    train_dataloaders=dm.train_dataloader(), 
                    val_dataloaders=dm.val_dataloader())

        # print training/validation metrics
        print(trainer.logged_metrics)

        # take the epoch with the best validation checkpoint, test it
        trainer.test(model=model, 
                     dataloaders=dm.test_dataloader(), 
                     ckpt_path='best')

        # store test accuracy for this fold (just one number)
        fold_accs.append(trainer.logged_metrics['test_acc'])
    
    # store fold accuracies for this iteration (n_fold numbers)
    iter_accs.append(fold_accs)

    # ensure save folder exists
    os.makedirs(os.path.join(acc_dir, context_prefix), exist_ok=True)

    # save partial results (up to this iteration) as CSV
    with open(os.path.join(acc_dir, f"{context_prefix}/{patient_id}_{context_prefix}_seq2seq_rnn_accs_iter{i+1}.csv"), 'w') as f:
        writer = csv.writer(f)
        writer.writerows(iter_accs)

    # print average accuracy across folds for this iteration
    print(np.mean(fold_accs))

# after all iterations: save final results
print(iter_accs)

# save all accuracies into one CSV
with open(os.path.join(acc_dir, f"{context_prefix}/{patient_id}_{context_prefix}_seq2seq_rnn_accs.csv"), 'w') as f:
    writer = csv.writer(f)
    writer.writerows(iter_accs)

# also save as .npy (easy reload in Python)
np.save(os.path.join(acc_dir, f"{context_prefix}/{patient_id}_{context_prefix}_seq2seq_rnn_accs.npy"),
        np.array(iter_accs))

##### Setting up data module for iteration 1 #####
DEBUG labels.shape: torch.Size([144, 3])
DEBUG first 5 rows of labels:
 tensor([[8, 0, 4],
        [7, 1, 4],
        [2, 5, 3],
        [1, 7, 0],
        [1, 8, 0]])


KeyboardInterrupt: 